# FL4ME: Centralized vs Federated (orchestration)

This notebook launches centralized and federated experiments via existing entry points:
- Centralized: `python train_central.py`
- Federated: `flwr run .`

Metrics tracked in W&B:
- Centralized: `train_loss`, `test_loss`, `test_acc` (already logged by `train_central.py`)
- Federated: depends on your Flower app (ensure it logs to the same W&B project/group)

Recommended seeds for a seminar: 3 (e.g., `[0, 1, 2]`). Use 5 if time permits for tighter confidence intervals.

In [ ]:
import os, sys, subprocess, shutil, time
import torch

# W&B configuration
def load_wandb_config():
    """Load WandB credentials from config file (optional for reproducibility)"""
    config = {}
    config_path = './.wandb_config'
    try:
        with open(config_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    config[key] = value
        return config.get('WANDB_ENTITY'), config.get('WANDB_PROJECT')
    except FileNotFoundError:
        return None, None

entity, project_name = load_wandb_config()
WANDB_MODE = "online"               # set to "offline" or "disabled" if needed

SEEDS = list(range(0, 6)) 

# Centralized hyperparameters
EPOCHS_CENTRAL = 30
LR_CENTRAL = 1e-3
EVAL_INTERVAL = 1

# Device
USE_GPU_FLAG = torch.cuda.is_available()
print("Device:", "cuda" if USE_GPU_FLAG else "cpu")
print("Seeds:", SEEDS)

def run_cmd(cmd_list, extra_env=None, cwd=None):
    env = os.environ.copy()
    if extra_env:
        env.update({k: str(v) for k, v in extra_env.items()})
    print("\n>>>", " ".join(cmd_list))
    print("    in:", cwd or os.getcwd())
    print("    with env overrides:", extra_env or {})
    proc = subprocess.Popen(cmd_list, cwd=cwd, env=env)
    proc.wait()
    if proc.returncode != 0:
        raise RuntimeError(f"Command failed with exit code {proc.returncode}: {' '.join(cmd_list)}")

Device: cpu
Seeds: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


## Centralized experiments

Runs `train_central.py` once per seed. The script already logs: `train_loss`, `test_loss`, `test_acc`.

Note: `train_central.py` does not expose a `--seed` flag. If you need strict reproducibility per seed, add seeding to the script; otherwise each run will use random initialization.

In [81]:
for seed in SEEDS:
    # W&B grouping so runs are easy to compare
    env_overrides = {
        "WANDB_PROJECT": project_name,
        "WANDB_MODE": WANDB_MODE,
        "WANDB_RUN_GROUP": "centralized",
        "WANDB_TAGS": "centralized,notebook",
        "WANDB_ENTITY": entity,

        # Optional: many apps read SEED from env; train_central.py currently does not
        "SEED": seed,
    }

    cmd = [sys.executable, "train_central.py",
           "--epochs", str(EPOCHS_CENTRAL),
           "--lr", str(LR_CENTRAL),
           "--eval-interval", str(EVAL_INTERVAL),
           "--wandb-mode", WANDB_MODE]
    if USE_GPU_FLAG:
        cmd.append("--gpu")

    print(f"\n=== Centralized run, seed={seed} ===")
    run_cmd(cmd, extra_env=env_overrides, cwd=os.getcwd())
    time.sleep(1)  # small gap between runs


=== Centralized run, seed=0 ===

>>> /Users/zoe/Desktop/Fed-CAD/.venv/bin/python train_central.py --epochs 30 --lr 0.001 --eval-interval 1 --wandb-mode online
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FL4ME', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'centralized', 'WANDB_TAGS': 'centralized,notebook', 'WANDB_ENTITY': 'louisewiljander-ludwig-maximilianuniversity-of-munich', 'SEED': 0}


wandb: Currently logged in as: zoematr (louisewiljander-ludwig-maximilianuniversity-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Traceback (most recent call last):
  File "/Users/zoe/Desktop/Fed-CAD/train_central.py", line 222, in <module>
    train_central(args)
  File "/Users/zoe/Desktop/Fed-CAD/train_central.py", line 71, in train_central
    wandb.init(
  File "/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/wandb/sdk/wandb_init.py", line 1586, in init
    run = wi.init(run_settings, run_config, run_printer)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/wandb/sdk/wandb_init.py", line 930, in init
    for module_name in telemetry.list_telemetry_imports(only_imported=True):
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/wandb/sdk/lib/telemetry.py", line 90, 

KeyboardInterrupt: 

In [ ]:
import os

# Check current state
print("1. Current directory:", os.getcwd())
print("2. pyproject.toml exists:", os.path.exists("pyproject.toml"))
print("3. Full path:", os.path.abspath("pyproject.toml"))

# List files to confirm
print("\n4. Files in current directory:")
for f in os.listdir("."):
    print(f"   {f}")

## Federated experiments (Flower Simulation Engine)

Runs `flwr run .` once per seed. Ensure your Flower app logs to W&B (same project) with metrics comparable to centralized (e.g., `global_train_loss`, `global_test_loss`, `global_test_acc`).

Tip: If your Flower app reads `WANDB_*` env vars and `SEED`, the grouping and seeds below will apply.

In [84]:
def run_flower(extra_env=None):
    # Prefer CLI if available; fallback to `python -m flwr`
    if shutil.which("flwr"):
        run_cmd(["flwr", "run", "."], extra_env=extra_env, cwd=os.getcwd())
    else:
        run_cmd([sys.executable, "-m", "flwr", "run", "."], extra_env=extra_env, cwd=os.getcwd())

entity, project_name = load_wandb_config()

for seed in SEEDS:
    env_overrides = {
        "WANDB_PROJECT": project_name,
        "WANDB_MODE": WANDB_MODE,
        "WANDB_RUN_GROUP": "federated",
        "WANDB_TAGS": "federated,notebook",
        "WANDB_ENTITY": entity,
        "SEED": seed,
    }
    print(f"\n=== Federated run (Flower), seed={seed} ===")
    run_flower(extra_env=env_overrides)
    time.sleep(1)


=== Federated run (Flower), seed=0 ===

>>> flwr run .
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FL4ME', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'federated', 'WANDB_TAGS': 'federated,notebook', 'WANDB_ENTITY': 'louisewiljander-ludwig-maximilianuniversity-of-munich', 'SEED': 0}
Loading project configuration... 
Success


wandb: Currently logged in as: zoematr (louisewiljander-ludwig-maximilianuniversity-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/wandb/analytics/sentry.py:279: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}
wandb: setting up run 3fhze1ag
/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/wandb/analytics/sentry.py:279: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /Users/zoe/Desktop/Fed-CAD/wandb/run-20260224_162637-3fhze1ag
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run federated_lr0.010_r10_le3
wandb: ⭐️ View project at https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FL4ME
wandb: 🚀 View run at htt


Final Results | test_loss=0.413 | test_acc=0.827
Metrics: prec=0.821, rec=0.827, f1=0.824, auc=0.849
Total training time: 0.23 minutes
Results saved to results.csv

Saving final model to disk...


wandb: updating run metadata
wandb: uploading config.yaml
wandb: 
wandb: Run summary:
wandb:           final_auc_roc 0.8489
wandb:                final_f1 0.8235
wandb:         final_precision 0.82123
wandb:            final_recall 0.82727
wandb:          final_test_acc 0.82727
wandb:         final_test_loss 0.41255
wandb:      total_local_epochs 30
wandb:            total_rounds 10
wandb: total_training_time_min 0.22525
wandb: 
wandb: 🚀 View run federated_lr0.010_r10_le3 at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FL4ME/runs/3fhze1ag
wandb: ⭐️ View project at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FL4ME
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20260224_162637-3fhze1ag/logs



=== Federated run (Flower), seed=1 ===

>>> flwr run .
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FL4ME', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'federated', 'WANDB_TAGS': 'federated,notebook', 'WANDB_ENTITY': 'louisewiljander-ludwig-maximilianuniversity-of-munich', 'SEED': 1}
Loading project configuration... 
Success


...

wandb: uploading config.yaml
wandb: 
wandb: Run summary:
wandb:           final_auc_roc 0.85348
wandb:                final_f1 0.80941
wandb:         final_precision 0.8164
wandb:            final_recall 0.82727
wandb:          final_test_acc 0.82727
wandb:         final_test_loss 0.39173
wandb:      total_local_epochs 30
wandb:            total_rounds 10
wandb: total_training_time_min 0.20154
wandb: 
wandb: 🚀 View run federated_lr0.010_r10_le3 at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FL4ME/runs/d0pl1yx4
wandb: ⭐️ View project at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FL4ME
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20260224_163059-d0pl1yx4/logs



=== Federated run (Flower), seed=11 ===

>>> flwr run .
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FL4ME', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'federated', 'WANDB_TAGS': 'federated,notebook', 'WANDB_ENTITY': 'louisewiljander-ludwig-maximilianuniversity-of-munich', 'SEED': 11}
Loading project configuration... 
Success


wandb: Currently logged in as: zoematr (louisewiljander-ludwig-maximilianuniversity-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/wandb/analytics/sentry.py:279: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}
wandb: setting up run f73czg4s
/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/wandb/analytics/sentry.py:279: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /Users/zoe/Desktop/Fed-CAD/wandb/run-20260224_163124-f73czg4s
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run federated_lr0.010_r10_le3
wandb: ⭐️ View project at https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FL4ME
wandb: 🚀 View run at htt


Final Results | test_loss=0.451 | test_acc=0.791
Metrics: prec=0.794, rec=0.791, f1=0.792, auc=0.823
Total training time: 0.22 minutes
Results saved to results.csv

Saving final model to disk...


wandb: uploading wandb-summary.json
wandb: uploading summary, console lines 1-140
wandb: 
wandb: Run summary:
wandb:           final_auc_roc 0.8228
wandb:                final_f1 0.79226
wandb:         final_precision 0.79374
wandb:            final_recall 0.79091
wandb:          final_test_acc 0.79091
wandb:         final_test_loss 0.4507
wandb:      total_local_epochs 30
wandb:            total_rounds 10
wandb: total_training_time_min 0.21692
wandb: 
wandb: 🚀 View run federated_lr0.010_r10_le3 at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FL4ME/runs/f73czg4s
wandb: ⭐️ View project at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FL4ME
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20260224_163124-f73czg4s/logs
